# K-Means Clustering Algorithms Guide
This notebook provides a comprehensive guide to using the k-means clustering algorithms implemented in HoneyTribe.
## Overview
The following algorithms are covered:
1. **WeightedKMeans** - Standard weighted K-means clustering
2. **InverseWeightedKMeans** - Inverse weighted variant with configurable parameters
3. **OnlineKMeans** - Online learning variant with incremental updates
4. **InverseWeightedOnlineKMeansV1** - Online variant with inverse weighting (v1)
5. **InverseWeightedOnlineKMeansV2** - Online variant with inverse weighting (v2)
6. **KHarmonicMeans** - Harmonic means variant

## Contents
1. Installation and imports
2. Creating sample data
3. Using each algorithm
4. Comparing algorithm performance
5. Visualization and interpretation
6. Advanced usage patterns

## 1. Installation and Imports

In [ ]:
# Install required packages (if needed)
!pip install numpy scikit-learn matplotlib pandas
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
warnings.filterwarnings('ignore')
# Import k-means algorithms
from honeytribe.taxonomy.k_means import (
    WeightedKMeans,
    InverseWeightedKMeans,
    OnlineKMeans,
    InverseWeightedOnlineKMeansV1,
    InverseWeightedOnlineKMeansV2,
    KHarmonicMeans,
)
print("✓ All imports successful!")
print(f"NumPy version: {np.__version__}")

## 2. Creating Sample DataLet's create synthetic data for demonstrating the algorithms.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
# Create synthetic data with 3 clusters
n_samples_per_cluster = 50
n_clusters = 3
# Cluster 1: centered at (0, 0)
cluster1 = np.random.randn(n_samples_per_cluster, 2) * 0.5 + np.array([0, 0])
# Cluster 2: centered at (5, 5)
cluster2 = np.random.randn(n_samples_per_cluster, 2) * 0.5 + np.array([5, 5])
# Cluster 3: centered at (2, 7)
cluster3 = np.random.randn(n_samples_per_cluster, 2) * 0.5 + np.array([2, 7])
# Combine all clusters
X = np.vstack([cluster1, cluster2, cluster3])
# Create true labels for comparison
y_true = np.array([0] * n_samples_per_cluster +
                 [1] * n_samples_per_cluster +
                 [2] * n_samples_per_cluster)

permutation = np.random.permutation(X.shape[0])

X = X[permutation]
y_true = y_true[permutation]

print(f"Data shape: {X.shape}")
print(f"Number of clusters: {n_clusters}")
print(f"Samples per cluster: {n_samples_per_cluster}")
# Visualize the data
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
plt.colorbar(scatter, label='True Cluster')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Synthetic Data with 3 Clusters')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print(f"✓ Data created successfully!")

## 3. Using Individual Algorithms
### 3.1 WeightedKMeans

In [ ]:
# Initialize and fit WeightedKMeans
weighted_kmeans = WeightedKMeans(n_clusters=3, max_iter=100, tol=1e-4)
weighted_kmeans.fit(X)
# Make predictions
labels_weighted = weighted_kmeans.predict(X)
# Get cluster centers
centers_weighted = weighted_kmeans.cluster_centers_
# Compute score
score_weighted = weighted_kmeans.score(X)
print("WeightedKMeans Results:")
print(f"  Score: {score_weighted:.4f}")
print(f"  Cluster centers shape: {centers_weighted.shape}")
print(f"  Unique labels: {np.unique(labels_weighted)}")
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# Plot 1: True labels
axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].set_title('True Labels')
axes[0].grid(True, alpha=0.3)
# Plot 2: Predicted labels
scatter = axes[1].scatter(X[:, 0], X[:, 1], c=labels_weighted, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
axes[1].scatter(centers_weighted[:, 0], centers_weighted[:, 1], c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].set_title('WeightedKMeans Predictions')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.2 InverseWeightedKMeans

In [ ]:
# Initialize and fit InverseWeightedKMeans
# The n and p parameters control the algorithm behavior
# Constraint: n >= p and n <= p + 2
iwk = InverseWeightedKMeans(n_clusters=3, max_iter=100, tol=1e-4, n=3, p=1)
iwk.fit(X)
labels_iwk = iwk.predict(X)
centers_iwk = iwk.cluster_centers_
score_iwk = iwk.score(X)
print("InverseWeightedKMeans Results:")
print(f"  Score: {score_iwk:.4f}")
print(f"  Parameters: n={iwk.n}, p={iwk.p}")
print(f"  Cluster centers shape: {centers_iwk.shape}")
# Visualize results
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels_iwk, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
plt.scatter(centers_iwk[:, 0], centers_iwk[:, 1], c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
plt.colorbar(scatter, label='Cluster')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('InverseWeightedKMeans Clustering')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.3 OnlineKMeans (Incremental Learning)

In [ ]:
# OnlineKMeans supports incremental learning
online_kmeans = OnlineKMeans(n_clusters=3, learning_rate=0.1, n_iter=5)
# Method 1: Fit with entire dataset at once
online_kmeans.fit(X)
labels_online = online_kmeans.predict(X)
centers_online = online_kmeans.cluster_centers_
score_online = online_kmeans.score(X)
print("OnlineKMeans Results:")
print(f"  Score: {score_online:.4f}")
print(f"  Learning rate: {online_kmeans.learning_rate}")
# Visualize results
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels_online, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
plt.scatter(centers_online[:, 0], centers_online[:, 1], c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
plt.colorbar(scatter, label='Cluster')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('OnlineKMeans Clustering')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.4 OnlineKMeans with Incremental Updates (Streaming Data)

In [ ]:
# OnlineKMeans also supports incremental learning via partial_fit
# This is useful for streaming data or mini-batch learning
online_kmeans_incremental = OnlineKMeans(n_clusters=3, learning_rate=0.05, n_iter=1)
# Simulate streaming data: process in batches
batch_size = 10
n_batches = len(X) // batch_size
scores_history = []
for batch_idx in range(n_batches):
    # Get batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    batch = X[start_idx:end_idx]
    # Update model
    online_kmeans_incremental.partial_fit(batch)
    # Compute score on entire dataset
    score = online_kmeans_incremental.score(X)
    scores_history.append(score)
    labels_incremental = online_kmeans_incremental.predict(X)
    print("OnlineKMeans Incremental Learning Results:")
    print(f"  Final score: {scores_history[-1]:.4f}")
    print(f"  Number of batches processed: {n_batches}")
    print(f"  Score improvement: {abs(scores_history[-1] - scores_history[0]):.4f}")
    # Plot convergence
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    # Plot 1: Score over batches
    axes[0].plot(scores_history, 'b-o', linewidth=2, markersize=6)
    axes[0].set_xlabel('Batch Number')
    axes[0].set_ylabel('Score')
    axes[0].set_title('OnlineKMeans: Score Convergence Over Batches')
    axes[0].grid(True, alpha=0.3)
    # Plot 2: Final clustering
    scatter = axes[1].scatter(X[:, 0], X[:, 1], c=labels_incremental, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
    axes[1].scatter(online_kmeans_incremental.cluster_centers_[:, 0],
                    online_kmeans_incremental.cluster_centers_[:, 1],
                    c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
    axes[1].set_xlabel('Feature 1')
    axes[1].set_ylabel('Feature 2')
    axes[1].set_title('Final OnlineKMeans Clustering')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 3.5 InverseWeightedOnlineKMeansV2

In [ ]:
# InverseWeightedOnlineKMeansV2 - improved online variant
iw_online_v2 = InverseWeightedOnlineKMeansV2(n_clusters=3, learning_rate=0.01, n_iter=5, n=2, p=2)
iw_online_v2.fit(X)
labels_v2 = iw_online_v2.predict(X)
centers_v2 = iw_online_v2.cluster_centers_
score_v2 = iw_online_v2.score(X)
print("InverseWeightedOnlineKMeansV2 Results:")
print(f"  Score: {score_v2:.4f}")
print(f"  Parameters: n={iw_online_v2.n}, p={iw_online_v2.p}")
print(f"  Learning rate: {iw_online_v2.learning_rate}")
# Visualize
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels_v2, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
plt.scatter(centers_v2[:, 0], centers_v2[:, 1], c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
plt.colorbar(scatter, label='Cluster')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('InverseWeightedOnlineKMeansV2 Clustering')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.6 KHarmonicMeans

In [ ]:
# K-Harmonic Means - robust clustering algorithm
khm = KHarmonicMeans(n_clusters=3, learning_rate=0.0001, n_iter=5)
khm.fit(X)
labels_khm = khm.predict(X)
centers_khm = khm.cluster_centers_
score_khm = khm.score(X)
print("KHarmonicMeans Results:")
print(f"  Score: {score_khm:.4f}")
print(f"  Learning rate: {khm.learning_rate}")
# Visualize
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=labels_khm, cmap='viridis', s=100, alpha=0.6, edgecolors='k')
plt.scatter(centers_khm[:, 0], centers_khm[:, 1], c='red', marker='X', s=300, edgecolors='black', linewidth=2, label='Centers')
plt.colorbar(scatter, label='Cluster')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('KHarmonicMeans Clustering')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Comparing Algorithm PerformanceLet's compare all algorithms on the same dataset.

In [ ]:
# Dictionary to store resultsresults = {}# 1. WeightedKMeanswk = WeightedKMeans(n_clusters=3, max_iter=100)wk.fit(X)results['WeightedKMeans'] = {    'score': wk.score(X),    'labels': wk.predict(X),    'centers': wk.cluster_centers_},# 2. InverseWeightedKMeansiwk = InverseWeightedKMeans(n_clusters=3, max_iter=100)iwk.fit(X)results['InverseWeightedKMeans'] = {    'score': iwk.score(X),    'labels': iwk.predict(X),    'centers': iwk.cluster_centers_},# 3. OnlineKMeansokm = OnlineKMeans(n_clusters=3, learning_rate=0.1)okm.fit(X)results['OnlineKMeans'] = {    'score': okm.score(X),    'labels': okm.predict(X),    'centers': okm.cluster_centers_},# 4. InverseWeightedOnlineKMeansV2iw_v2 = InverseWeightedOnlineKMeansV2(n_clusters=3, learning_rate=0.05)iw_v2.fit(X)results['IWOnlineKMeansV2'] = {    'score': iw_v2.score(X),    'labels': iw_v2.predict(X),    'centers': iw_v2.cluster_centers_},# 5. KHarmonicMeanskhm = KHarmonicMeans(n_clusters=3, learning_rate=0.05)khm.fit(X)results['KHarmonicMeans'] = {    'score': khm.score(X),    'labels': khm.predict(X),    'centers': khm.cluster_centers_},# Create comparison tablecomparison_df = pd.DataFrame({    'Algorithm': list(results.keys()),    'Score': [results[alg]['score'] for alg in results.keys()]})comparison_df = comparison_df.sort_values('Score', ascending=False)print("Algorithm Performance Comparison:")print(comparison_df.to_string(index=False))print(f"Best algorithm: {comparison_df.iloc[0]['Algorithm']} (Score: {comparison_df.iloc[0]['Score']:.4f})")

### Visualization: Compare All Algorithms

In [ ]:
# Create a grid of subplots for all algorithmsfig, axes = plt.subplots(2, 3, figsize=(16, 10))axes = axes.flatten()algorithm_names = list(results.keys())for idx, (ax, alg_name) in enumerate(zip(axes, algorithm_names)):    labels = results[alg_name]['labels']    centers = results[alg_name]['centers']    score = results[alg_name]['score']        # Scatter plot    scatter = ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=80, alpha=0.6, edgecolors='k')        # Plot centers    ax.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=250, edgecolors='black', linewidth=1.5)        ax.set_xlabel('Feature 1', fontsize=10)    ax.set_ylabel('Feature 2', fontsize=10)    ax.set_title(f'{alg_name}Score: {score:.2f}', fontsize=11, fontweight='bold')    ax.grid(True, alpha=0.3)# Hide the last subplotaxes[-1].axis('off')plt.suptitle('K-Means Algorithm Comparison', fontsize=14, fontweight='bold', y=0.995)plt.tight_layout()plt.show()

### Score Comparison Bar Chart

In [ ]:
# Bar chart comparing scoresfig, ax = plt.subplots(figsize=(12, 6))algorithms = list(results.keys())scores = [results[alg]['score'] for alg in algorithms]colors = plt.cm.viridis(np.linspace(0, 1, len(algorithms)))bars = ax.bar(algorithms, scores, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)# Add value labels on barsfor bar, score in zip(bars, scores):    height = bar.get_height()    ax.text(bar.get_x() + bar.get_width()/2., height,            f'{score:.2f}',            ha='center', va='bottom', fontsize=10, fontweight='bold')ax.set_ylabel('Score (Negative Sum of Squared Distances)', fontsize=11)ax.set_xlabel('Algorithm', fontsize=11)ax.set_title('Algorithm Performance Comparison', fontsize=12, fontweight='bold')ax.grid(True, alpha=0.3, axis='y')plt.xticks(rotation=45, ha='right')plt.tight_layout()plt.show()print("Score Interpretation:")print("- Higher scores (less negative) indicate better clustering")print("- Score = -sum of squared distances to nearest cluster center")

## 5. Advanced Usage Patterns
### 5.1 Trying Different Numbers of Clusters

In [ ]:
# Elbow method: find optimal number of clusters
cluster_range = range(1, 8)
scores_by_clusters = {}
for n_clusters in cluster_range:
    kmeans = WeightedKMeans(n_clusters=n_clusters, max_iter=100)
    kmeans.fit(X)
    score = kmeans.score(X)
    scores_by_clusters[n_clusters] = score
    print(f"n_clusters={n_clusters}: score={score:.4f}")

# Plot the elbow curve
plt.figure(figsize=(10, 6))
clusters = list(scores_by_clusters.keys())
scores = list(scores_by_clusters.values())
plt.plot(clusters, scores, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=3, color='r', linestyle='--', linewidth=2, label='True number of clusters')
plt.xlabel('Number of Clusters', fontsize=11)
plt.ylabel('Score', fontsize=11)
plt.title('Elbow Method: Finding Optimal Number of Clusters', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()
print(f"Optimal number of clusters (by elbow method): ~3")

### 5.2 Effect of Learning Rate (Online Algorithms)

In [ ]:
# Compare different learning rates
learning_rates = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]
scores_by_lr = {}
for lr in learning_rates:
    kmeans = OnlineKMeans(n_clusters=3, learning_rate=lr, n_iter=10)
    kmeans.fit(X)
    score = kmeans.score(X)
    scores_by_lr[lr] = score
    print(f"learning_rate={lr}: score={score:.4f}")
# Plot learning rate effect
plt.figure(figsize=(10, 6))
lrs = list(scores_by_lr.keys())
scores = list(scores_by_lr.values())
plt.plot(lrs, scores, 'go-', linewidth=2, markersize=8)
plt.xlabel('Learning Rate', fontsize=11)
plt.ylabel('Score', fontsize=11)
plt.title('Effect of Learning Rate on OnlineKMeans Performance', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3 Effect of Parameters (InverseWeightedKMeans)

In [ ]:
# Compare different n, p parameter combinations# Constraint: n >= p and n <= p + 2param_combinations = [(1, 1), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]scores_by_params = {}for n, p in param_combinations:    if n >= p and n <= p + 2:  # Check constraints        kmeans = InverseWeightedKMeans(n_clusters=3, max_iter=100, n=n, p=p)        kmeans.fit(X)        score = kmeans.score(X)        scores_by_params[(n, p)] = score        print(f"n={n}, p={p}: score={score:.4f}")# Visualize parameter effectsfig, ax = plt.subplots(figsize=(12, 6))param_labels = [f'n={n}, p={p}' for n, p in scores_by_params.keys()]param_scores = list(scores_by_params.values())colors = plt.cm.viridis(np.linspace(0, 1, len(param_labels)))bars = ax.bar(param_labels, param_scores, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)# Add value labelsfor bar, score in zip(bars, param_scores):    height = bar.get_height()    ax.text(bar.get_x() + bar.get_width()/2., height,            f'{score:.2f}',            ha='center', va='bottom', fontsize=9, fontweight='bold')ax.set_ylabel('Score', fontsize=11)ax.set_xlabel('Parameters (n, p)', fontsize=11)ax.set_title('InverseWeightedKMeans: Effect of Parameters n and p', fontsize=12, fontweight='bold')ax.grid(True, alpha=0.3, axis='y')plt.xticks(rotation=45, ha='right')plt.tight_layout()plt.show()

### 5.4 Handling Higher-Dimensional Data

In [ ]:
# Create higher-dimensional data (10D)
np.random.seed(42)
n_features = 1000
n_samples_per_cluster = 50
cluster1_hd = np.random.randn(n_samples_per_cluster, n_features) * 0.5 + 0
cluster2_hd = np.random.randn(n_samples_per_cluster, n_features) * 0.5 + 5
cluster3_hd = np.random.randn(n_samples_per_cluster, n_features) * 0.5 + 3
X_hd = np.vstack([cluster1_hd, cluster2_hd, cluster3_hd])
y_true_hd = np.array([0] * n_samples_per_cluster + [1] * n_samples_per_cluster + [2] * n_samples_per_cluster)

permutation = np.random.permutation(X.shape[0])

X_hd = X_hd[permutation]
y_true_hd = y_true_hd[permutation]

print(f"High-dimensional data shape: {X_hd.shape}")
# Test all algorithms on high-dimensional data
hd_results = {}
algorithms_hd = [
    ('WeightedKMeans', WeightedKMeans(n_clusters=3, max_iter=1000)),
    ('InverseWeightedKMeans', InverseWeightedKMeans(n_clusters=3, max_iter=1000)),
    ('OnlineKMeans', OnlineKMeans(n_clusters=3, learning_rate=0.1, n_iter=10)),
    ('InverseWeightedOnlineKMeansV2', InverseWeightedOnlineKMeansV2(n_clusters=3, learning_rate=0.05)),
    ('KHarmonicMeans', KHarmonicMeans(n_clusters=3, learning_rate=0.05)),
]

for name, model in algorithms_hd:
    model.fit(X_hd)
    score = model.score(X_hd)
    hd_results[name] = score
    print(f"{name}: {score:.4f}")

# Plot results
fig, ax = plt.subplots(figsize=(12, 6))
alg_names = list(hd_results.keys())
alg_scores = list(hd_results.values())
colors = plt.cm.viridis(np.linspace(0, 1, len(alg_names)))
bars = ax.bar(alg_names, alg_scores, color=colors, edgecolor='black', linewidth=1.5, alpha=0.8)
for bar, score in zip(bars, alg_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{score:.2f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')
ax.set_ylabel('Score', fontsize=11)
ax.set_title(f'Algorithm Performance on High-Dimensional Data ({n_features}D)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 6. Key Takeaways and Best Practices
### Algorithm Selection Guide
| Algorithm | Use Case | Pros | Cons |
|-----------|----------|------|------|
| **WeightedKMeans** | General clustering | Balances local and global interactions | May not find global optimum |
| **InverseWeightedKMeans** | Inverse distance weighting | Configurable parameters (n, p) | More complex to tune |
| **OnlineKMeans** | Streaming/incremental data | Efficient for large datasets | Requires learning rate tuning |
| **InverseWeightedOnlineKMeansV2** | Online with inverse weighting | Combines online and weighting | More computation per step |
| **KHarmonicMeans** | Robust clustering | Less sensitive to initialization | More complex algorithm |
### Best Practices
1. **Data Preprocessing**
    - Normalize/standardize features to have similar scales
    - Remove outliers that might skew cluster centers
    - Handle missing values appropriately
2. **Choosing Number of Clusters**
    - Use the elbow method
    - Try multiple values and compare results
    - Use domain knowledge if available
3. **Algorithm Selection**
    - Start with WeightedKMeans for simplicity
    - Use OnlineKMeans for large/streaming datasets
    - Try multiple algorithms and compare scores
4. **Hyperparameter Tuning**
    - For online algorithms: adjust learning rate (0.01-0.5)
    - For InverseWeightedKMeans: tune n and p (constraint: n >= p, n <= p+2)
    - For batch algorithms: increase max_iter if needed
5. **Evaluation**
    - Compare scores across algorithms
    - Visualize results to ensure they make sense
    - Use domain expertise to validate clustering
### Key Parameters
```python
# Common parameters across algorithmsn_clusters
# Number of clusters (required)
max_iter            # Max iterations for batch algorithms
learning_rate       # Step size for online algorithms (default: 0.05)
# Specific parameters
tol                 # Convergence tolerance (default: 1e-4)
n, p                # InverseWeighted parameters (constraints: n >= p, n <= p+2)
```